refer to this for installation https://github.com/abetlen/llama-cpp-python#installation-with-openblas--cublas--clblast--metal 

In [2]:
# Uncomment to install dependencies
%pip install llama_index
%pip install llama-cpp-python
%pip install transformers
%pip install torch
%pip install tqdm


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import logging
import sys

from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)  # Change INFO to DEBUG if you want more extensive logging
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

llm = LlamaCPP(
    model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf",
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 4}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [3]:
# Create an index of your documents
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

storage_directory = "./storage"

documents = SimpleDirectoryReader('./testdata').load_data()

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)


index = VectorStoreIndex.from_documents(documents, service_context=service_context)
# Persist the index to disk
display(index)
index.storage_context.persist(persist_dir=storage_directory)

c:\Users\User\Downloads\Github\MIMI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**********
Trace: index_construction
    |_node_parsing ->  0.062839 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.000998 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.000999 seconds
      |_chunking ->  0.000996 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.000998 seconds
      |_chunking ->  0.000992 seconds
      |_chunking ->  0.001996 seconds
      |_chunking ->  0.001002 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.000996 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.008005 seconds
      |_chunking ->  0.005993 seconds
      |_chunking ->  0.003984 seconds
      |_chunking ->  0.004996 seconds
      |_chunking ->  0.006949 seconds
      |_chunking ->  0.004987 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.001993 seconds
      |_chunking ->  0.000998 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.00399 seconds
      |_chunking ->  0.0009

In [4]:
# Now you can load the index from disk when needed, and not rebuild it each time.
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import StorageContext, load_index_from_storage

# transcript_directory = "transcripts/ancient-aliens-official"
storage_directory = "./storage"

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)

storage_context = StorageContext.from_defaults(persist_dir=storage_directory)
index = load_index_from_storage(storage_context, service_context=service_context)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
**********
Trace: index_construction
**********


In [5]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

# response = query_engine.query("What do you think of Facebook's LLaMa?")
# print(response)

# query = "What is the Alien nuclear agenda? \n\n" \
#         "Please summarize the information into 3 detailed paragraphs, something suitable for a blog post."

query = "Samsui women or Coolie more important for Singapore's Development? \n\n" \
        "Please summarize the information into 3 sentences. Something shorter to read"

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

**********
Trace: query
    |_query ->  470.860184 seconds
      |_retrieve ->  0.030889 seconds
        |_embedding ->  0.029888 seconds
      |_synthesize ->  470.827865 seconds
        |_templating ->  0.0 seconds
        |_llm ->  470.792545 seconds
**********


<b>  Sure, I'd be happy to help! Based on the context information provided, it appears that Samsui women played a more significant role in Singapore's development than coolies. The women worked as manual laborers in construction and other industries, contributing to the country's growth and prosperity. In contrast, coolies were primarily used for their physical strength and were not as integral to Singapore's development as the Samsui women.</b>

In [6]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

# response = query_engine.query("What do you think of Facebook's LLaMa?")
# print(response)

# query = "What is the Alien nuclear agenda? \n\n" \
#         "Please summarize the information into 3 detailed paragraphs, something suitable for a blog post."

query = "What do you think of Facebook's LLaMa? \n\n" \
        "Please summarize the information into 3 sentences, something suitable for a tweet."

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit


**********
Trace: query
    |_query ->  387.776819 seconds
      |_retrieve ->  0.424012 seconds
        |_embedding ->  0.387111 seconds
      |_synthesize ->  387.352807 seconds
        |_templating ->  0.0 seconds
        |_llm ->  387.297411 seconds
**********


<b>  Based on the provided context, I can provide the following summary of Facebook's LLaMa:

LLaMa is a state-of-the-art foundational large language model designed to help researchers advance their work in the subfield of AI. It is trained on a large set of unlabeled data and is ideal for fine-tuning for a variety of tasks, making it a valuable resource for the research community. However, full research access to large language models like LLaMa remains limited due to the resources required to train and run such models, which hinders progress on efforts to improve their robustness and mitigate known issues.</b>

In [7]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

# Create a reader for Wikipedia articles
loader = WikipediaReader()
wikidocs = loader.load_data(pages=['Samsui', 'Coolie'])

display(wikidocs)


[Document(id_='65444221-3990-42c8-8499-f30d22648f8b', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='112d037b39cf02ff4ca88a4207c4518d2675ed5023a611f6da89acf7a44fdccf', text='Samsung Group, or simply Samsung (Korean: 삼성; RR: samseong [samsʌŋ]) (stylized as SΛMSUNG), is a South Korean multinational manufacturing conglomerate headquartered in Samsung Digital City, Suwon, South Korea. It comprises numerous affiliated businesses, most of them united under the Samsung brand, and is the largest South Korean chaebol (business conglomerate). As of 2020, Samsung has the eighth highest global brand value.Samsung was founded by Lee Byung-chul in 1938 as a trading company. Over the next three decades, the group diversified into areas including food processing, textiles, insurance, securities, and retail. Samsung entered the electronics industry in the late 1960s and the construction and shipbuilding industries in the mid-1970s; t